## Training script for the CNN 

Loads in the converted plane representation of the pgn files, defines the network architecture and starts the training process. Checkpoints of the weights are saved if there's an improvement in the validation loss.
The training performance metrics (e.g. losses, accuracies...) are exported to tensorboard and can be checked during training.
* author: QueensGambit

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
from __future__ import print_function
import sys
sys.path.insert(0,'../../../')
from DeepCrazyhouse.src.runtime.color_logger import enable_color_logging
import logging
import mxnet as mx
from mxnet import gluon
from mxnet import autograd as ag
import numpy as np
import matplotlib.pyplot as plt
from time import time
import datetime
import random
from DeepCrazyhouse.src.domain.crazyhouse.input_representation import board_to_planes, planes_to_board
from DeepCrazyhouse.src.domain.crazyhouse.output_representation import policy_to_moves, policy_to_best_move, policy_to_move
from DeepCrazyhouse.src.preprocessing.dataset_loader import load_pgn_dataset
import chess
import re
from time import time
from DeepCrazyhouse.src.domain.neural_net.architectures.a0_resnet import AlphaZeroResnet
from DeepCrazyhouse.src.domain.neural_net.architectures.rise import Rise
from DeepCrazyhouse.src.domain.neural_net.architectures.densenet import DenseNet
from DeepCrazyhouse.src.domain.neural_net.architectures.sparsenet import SparseNet
from DeepCrazyhouse.src.preprocessing.pgn_record_dataset import PGNRecordDataset
from DeepCrazyhouse.configs.main_config import main_config
from mxboard import SummaryWriter
from tqdm import tqdm_notebook
import os
from mxnet import nd, autograd
from collections import deque
from copy import deepcopy
from multiprocessing import cpu_count
from DeepCrazyhouse.configs.main_config import main_config
from DeepCrazyhouse.src.training.trainer_agent import TrainerAgent, evaluate_metrics, acc_sign, reset_metrics

from DeepCrazyhouse.src.training.lr_schedules.lr_schedules import *
import glob

enable_color_logging()
%matplotlib inline

/home/queensgambit/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


ImportError: /home/queensgambit/anaconda3/lib/python3.6/site-packages/google/protobuf/pyext/_message.cpython-36m-x86_64-linux-gnu.so: undefined symbol: _ZNK6google8protobuf10TextFormat17FieldValuePrinter9PrintBoolEb

## Settings

In [ ]:
# set the context on CPU, switch to GPU if there is one available (strongly recommended for training)
ctx = mx.gpu()
# set a specific seed value for reproducability
seed = 42

export_weights = True
log_metrics_to_tensorboard = True
export_grad_histograms = True

# batch_steps = 1000 means for example that every 1000 batches the validation set gets processed
batch_steps = 1000 # this defines how often a new checkpoint will be saved and the metrics evaluated
# k_steps_initial defines how many steps have been trained before
# (k_steps_initial != 0 if you continue training from a checkpoint)
k_steps_initial = 0
# this is the network based on human play
# symbol_file = 'model-1.32459-0.566-symbol.json'
# params_file = 'model-1.32459-0.566-0074.params'

batch_size = 1024 # the batch_size needed to be reduced to 1024 in order to fit in the GPU 1080Ti
#4096 was originally used in the paper -> works slower for current GPU
# 2048 was used in the paper Mastering the game of Go without human knowledge and fits in GPU memory
#typically if you half the batch_size, you should double the lr

# otimization parameters
max_lr = 0.35
min_lr = 0.00001
max_momentum = 0.95
min_momentum = 0.8
# loads a previous checkpoint if the loss increased significanly
use_spike_recovery = True
# stop training as soon as max_spikes has been reached
max_spikes = 20
# define spike threshold when the detection will be triggered
spike_thresh = 1.5
# weight decay
wd = 1e-4
# weight the value loss a lot lower than the policy loss in order to prevent overfitting
val_loss_factor = 0.5
policy_loss_factor = 0.5

normalize = True # define whether to normalize input data to [0,1]
nb_epochs = 7 # define how many epoches the network will be trained

In [5]:
# Fixing the random seed
mx.random.seed(seed)

### Create a ./logs and ./weights directory

In [6]:
!mkdir ./logs && mkdir ./weights

mkdir: cannot create directory ‘./logs’: File exists


### load the config file

In [7]:
print(main_config)

{'pgn_train_dir': '/home/demo_user/datasets/lichess/Crazyhouse/pgn/train/', 'pgn_val_dir': '/home/demo_user/datasets/lichess/Crazyhouse/pgn/val/', 'pgn_test_dir': '/home/demo_user/datasets/lichess/Crazyhouse/pgn/test/', 'pgn_mate_in_one_dir': '/home/demo_user/datasets/lichess/Crazyhouse/pgn/mate_in_one/', 'planes_train_dir': '/home/demo_user/datasets/lichess/Crazyhouse/planes/train/', 'planes_val_dir': '/home/demo_user/datasets/lichess/Crazyhouse/planes/val/', 'planes_test_dir': '/home/demo_user/datasets/lichess/Crazyhouse/planes/test/', 'planes_mate_in_one_dir': '/home/demo_user/datasets/lichess/Crazyhouse/planes/mate_in_one/', 'rec_dir': '/home/demo_user/datasets/lichess/Crazyhouse/rec/', 'model_architecture_dir': '/home/queensgambit/Programming/Deep_Learning/CrazyAra_0.2.0/model/symbol/', 'model_weights_dir': '/home/queensgambit/Programming/Deep_Learning/CrazyAra_0.2.0/model/params/'}


In [8]:
CPU_COUNT = cpu_count()
CPU_COUNT

8

### load the dataset-files

### Validation Dataset (which is used during training)

s_idcs_val, x_val, yv_val, yp_val, pgn_datasets_val = load_pgn_dataset(dataset_type='val', part_id=0,
                                                                           print_statistics=True, print_parameters=True, normalize=normalize)
val_dataset = gluon.data.ArrayDataset(nd.array(x_val), nd.array(yv_val), nd.array(yp_val.argmax(axis=1)))
val_data = gluon.data.DataLoader(val_dataset, batch_size, shuffle=False, num_workers=CPU_COUNT)

In [10]:
nb_parts = len(glob.glob(main_config['planes_train_dir'] + '**/*'))

nb_it_per_epoch = (len(x_val) * nb_parts) // batch_size # calculate how many iterations per epoch exist
# one iteration is defined by passing 1 batch and doing backprop
total_it = int(nb_it_per_epoch * nb_epochs)

### Define a Learning Rate schedule
* Here we apply a fine tuning schedule with a very low learning rate

lr_schedule = OneCycleSchedule(start_lr=max_lr/8, max_lr=max_lr, cycle_length=total_it*.3, cooldown_length=total_it*.6, finish_lr=min_lr)
lr_schedule = LinearWarmUp(lr_schedule, start_lr=min_lr, length=total_it/30)
plot_schedule(lr_schedule, iterations=total_it)

### Momentum schedule

momentum_schedule = MomentumSchedule(lr_schedule, min_lr, max_lr, min_momentum, max_momentum)
plot_schedule(momentum_schedule, iterations=total_it, ylabel='Momentum')

### Create the model

input_shape = x_val[0].shape

input_shape

yp_val.shape

In [18]:
try:
    del net
except:
    pass

### Load the pretrained model

In [19]:
# net = gluon.nn.SymbolBlock.imports(symbol_file='weights/%s'%symbol_file, input_names='data', param_file='weights/%s'%params_file, ctx=ctx)

In [20]:
net = AlphaZeroResnet(n_labels=2272, channels=256, channels_value_head=8, channels_policy_head=16, num_res_blocks=19, value_fc_size=256, bn_mom=0.9, act_type='relu')

In [21]:
#net = Rise(n_labels=yp_val.shape[1], channels=256, channels_value_head=8, channels_policy_head=16, nb_res_blocksX=7, nb_res_blocksX_neck=12, cardinality=1, cardinality_neck=2, value_fc_size=256, bn_mom=0.9, act_type='lrelu', use_se=True, res_scale_fac=0.2, use_rise_stem=False)

In [22]:
#net = DenseNet(n_init_features=64, growth_rate=16, n_layers=3, bn_size=4, dropout=0, n_labels=2272, channels_value_head=8, channels_policy_head=16, value_fc_size=256, bn_mom=0.9, act_type='lrelu',)

In [23]:
net = SparseNet(n_init_features=64, growth_rate=16, n_layers=40, bn_size=4, n_paths=14, n_labels=2272, channels_value_head=8, channels_policy_head=16, value_fc_size=256, bn_mom=0.9, act_type='lrelu',)

## Network summary

In [24]:
print(net)

SparseNet(
  (body): HybridSequential(
    (0): _SparseNetBlock(
      (body): HybridConcurrent(
        (0): Identity(
        
        )
        (1): HybridSequential(
          (0): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=None)
          (1): Activation(relu)
          (2): Conv2D(None -> 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (3): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=None)
          (4): Activation(relu)
          (5): Conv2D(None -> 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
    )
    (1): _SparseNetBlock(
      (body): HybridConcurrent(
        (0): Identity(
        
        )
        (1): HybridSequential(
          (0): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=None)
          (1): Activation(relu)
          (2): Conv2D(None -> 64, kernel_size=(1, 

mx.viz.plot_network(
    net(mx.sym.var('data'))[1], 
    shape={'data':(1, input_shape[0], input_shape[1], input_shape[2])},
    node_attrs={"shape":"oval","fixedsize":"false"}
)


In [25]:
input_shape = (34, 8, 8)

In [26]:
mx.viz.print_summary(
    net(mx.sym.var('data'))[1], 
    shape={'data':(1, input_shape[0], input_shape[1], input_shape[2])},
)

AssertionError: Argument indices must be Symbol instances, but got [<Symbol a>]

## Initialize the weights 
(only needed if no pretrained weights are used)

In [ ]:
net.collect_params().initialize(mx.init.Xavier(rnd_type='uniform', factor_type='avg', magnitude=2.24), ctx=ctx)
net.hybridize()

## Define the metrics to use

In [ ]:
metrics = {
    'value_loss': mx.metric.MSE(name='value_loss', output_names=['value_output']),
    'policy_loss': mx.metric.CrossEntropy(name='policy_loss', output_names=['policy_output'],
                                                label_names=['policy_label']),
    'value_acc_sign': mx.metric.create(acc_sign, name='value_acc_sign', output_names=['value_output'],
                                             label_names=['value_label']),
    'policy_acc': mx.metric.Accuracy(axis=1, name='policy_acc', output_names=['policy_output'],
                                           label_names=['policy_label'])
}

## Define a training agent

In [ ]:
train_agent = TrainerAgent(net, val_data, nb_parts, lr_schedule, momentum_schedule, total_it, wd=wd, batch_steps=batch_steps,
                 k_steps_initial=k_steps_initial, cpu_count=CPU_COUNT-3, batch_size=batch_size, normalize=normalize, export_weights=export_weights,
                 export_grad_histograms=export_grad_histograms, log_metrics_to_tensorboard=log_metrics_to_tensorboard, ctx=ctx, metrics=metrics,
                use_spike_recovery=use_spike_recovery, max_spikes=max_spikes, spike_thresh=spike_thresh, seed=seed, val_loss_factor=val_loss_factor, policy_loss_factor=policy_loss_factor)

## Performance Pre-Training

In [ ]:
evaluate_metrics(metrics, val_data, net, nb_batches=None)

## Start the training process

In [ ]:
(k_steps_final, val_loss_final, val_p_acc_final), (k_steps_best, val_loss_best, val_p_acc_best) = train_agent.train()

## Export the last model state

In [ ]:
# export the model via mxnet checkpoint
prefix ="./weights/model-%.5f-%.3f" % (val_loss_final, val_p_acc_final)
# the export function saves both the architecture and the weights
net.export(prefix, epoch=k_steps_final) 

In [ ]:
prefix

## Load the best model once again

In [ ]:
# delete the current net object form memory
del net

In [ ]:
model_prefix ="./weights/model-%.5f-%.3f" % (val_loss_best, val_p_acc_best)
model_arch_path = '%s-symbol.json' % model_prefix
model_params_path = '%s-%04d.params' % (model_prefix, k_steps_best)
print('load current best model:', model_params_path)
#net.load_parameters(model_path, ctx=ctx)
net = gluon.nn.SymbolBlock.imports(model_arch_path, ['data'], model_params_path, ctx=ctx)

In [ ]:
print('best val_loss: %.5f with v_policy_acc: %.5f at k_steps_best %d' % (val_loss_best, val_p_acc_best, k_steps_best))

In [ ]:
idx = 0

In [ ]:
board = planes_to_board(x_val[idx], normalized_input=normalize)

print(chess.COLOR_NAMES[board.turn])
print(board.pockets)
board

In [ ]:
def predict_single(net, x):
    
    out = [None, None]
    pred = net(mx.nd.array(np.expand_dims(x, axis=0), ctx=ctx))
    pred[1] = pred[1].softmax()
    out[0] = pred[0].asnumpy()
    out[1] = pred[1].asnumpy()
    
    return out

In [ ]:
pred = predict_single(net, x_val[0])
pred

In [ ]:
policy_to_best_move(board, yp_val[idx])

In [ ]:
opts = 5
selected_moves, probs = policy_to_moves(board, pred[1][0])
selected_moves[:opts]

In [ ]:
plt.barh(range(opts)[::-1], probs[:opts])
ax = plt.gca()
ax.set_yticks(range(opts)[::-1])
ax.set_yticklabels(selected_moves)

In [ ]:
board = chess.variant.CrazyhouseBoard()
board.push_uci('e2e4')
board.push_uci('e7e5')
board.push_uci('f1c4')
board.push_uci('b8c6')
board.push_uci('d1h5')
x_scholar_atck = board_to_planes(board, normalize=normalize)
board

In [ ]:
pred = predict_single(net, x_scholar_atck)

selected_moves, probs = policy_to_moves(board, pred[1][0])
plt.barh(range(opts)[::-1], probs[:opts])
ax = plt.gca()
ax.set_yticks(range(opts)[::-1])
ax.set_yticklabels(selected_moves)

In [ ]:
board.push(selected_moves[0])
board

### Performance on test dataset


In [ ]:
s_idcs_test, x_test, yv_test, yp_test, pgn_datasets_test = load_pgn_dataset(dataset_type='test', part_id=0,
                                                                           print_statistics=True, print_parameters=True, normalize=True)
test_dataset = gluon.data.ArrayDataset(nd.array(x_test), nd.array(yv_test), nd.array(yp_test.argmax(axis=1)))
test_data = gluon.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=CPU_COUNT)

In [ ]:
evaluate_metrics(metrics, test_data, net, nb_batches=None)

### Show result on mate-in-one problems

In [ ]:
s_idcs_mate, x_mate, yv_mate, yp_mate, pgn_dataset_mate = load_pgn_dataset(dataset_type='mate_in_one',
                                                         print_parameters=True, print_statistics=True, normalize=normalize)

In [ ]:
mate_dataset = mx.gluon.data.dataset.ArrayDataset(nd.array(x_mate), nd.array(yv_mate), nd.array(yp_mate.argmax(axis=1)))
mate_data = mx.gluon.data.DataLoader(mate_dataset, batch_size=batch_size, num_workers=CPU_COUNT)

### Mate In One Performance

In [ ]:
#mod_p.score(mate_iter, metrics)
evaluate_metrics(metrics, mate_data, net)

### Show some example mate problems

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#from DeepCrazyhouse.src.domain.crazyhouse.legal_move_filter import mirror_move

### Evaluate Performance

In [ ]:
def eval_pos(net, x_mate, yp_mate, verbose=False):
    
    board = planes_to_board(x_mate, normalized_input=normalize)
    if verbose is True:
        print("{0}'s turn".format(chess.COLOR_NAMES[board.turn]))
        print("black/white {0}".format(board.pockets))
    pred = predict_single(net, x_mate)
    
    
    #true_mat = yp_mate.reshape(78,8,8)
    #true_move = filter_moves(board, true_mat, is_white_to_move=board.turn, renormalize_confidences=False)[0][0]
    true_move = policy_to_move(yp_mate, is_white_to_move=board.turn)
    
    #pred_mat = pred[1][0].reshape(78,8,8)
    opts = 5
    pred_moves, probs = policy_to_moves(board, pred[1][0])
    pred_moves = pred_moves[:opts]
    
    #pred_moves = filter_moves(board, pred_mat, is_white_to_move=board.turn, renormalize_confidences=False)[0][:opts]

    legal_move_cnt = board.legal_moves.count()
    mate_move_cnt = str(board.legal_moves).count('#')
    
    is_mate_5_top = False
    
    for pred_move in pred_moves:
        board_5_top = deepcopy(board)
        board_5_top.push(pred_move)
        if board_5_top.is_checkmate() is True:
            is_mate_5_top = True
            break
    
    board.push(pred_moves[0])
    
    is_checkmate = False
    if board.is_checkmate() is True:
        is_checkmate = True
        
    filtered_pred = sorted(pred[1][0], reverse=True)
    
    if verbose is True:
        plt.barh(range(opts)[::-1], filtered_pred[:opts])
        ax = plt.gca()
        ax.set_yticks(range(opts)[::-1])
        ax.set_yticklabels(pred_moves)
        plt.title('True Move:' + str(true_move) +
                 '\nEval:' + str(pred[0][0]))
        plt.show()
    
    return pred, pred_moves, true_move, board, is_checkmate, is_mate_5_top, legal_move_cnt, mate_move_cnt

In [ ]:
nb_pos = len(x_mate)
mates_found = []
mates_5_top_found = []
legal_mv_cnts = []
mate_mv_cnts = []

for i in range(nb_pos):
    pred, pred_moves, true_move, board, is_mate, is_mate_5_top, legal_mv_cnt, mate_mv_cnt= eval_pos(net, x_mate[i], yp_mate[i])
    mates_found.append(is_mate)
    legal_mv_cnts.append(legal_mv_cnt)
    mate_mv_cnts.append(mate_mv_cnt)
    mates_5_top_found.append(is_mate_5_top)

In [ ]:
np.array(mate_mv_cnts).mean()

In [ ]:
np.array(legal_mv_cnts).mean()

### Random Guessing Baseline

In [ ]:
np.array(mate_mv_cnts).mean() / np.array(legal_mv_cnts).mean()

### Prediciton Performance

In [ ]:
print('mate_in_one_acc:', sum(mates_found) / nb_pos)

In [ ]:
sum(mates_5_top_found) / nb_pos

In [ ]:
pgn_dataset_mate.tree()

In [ ]:
metadata = np.array(pgn_dataset_mate['metadata'])
metadata[0, :]
metadata[1, :]

In [ ]:
site_mate = metadata[1:, 1]

In [ ]:
def clean_string(np_string):
    string = str(site_mate[i]).replace("b'", "")
    string = string.replace("'", "")
    string = string.replace('"', '')
    
    return string

In [ ]:
import chess.svg
from IPython.display import SVG, HTML

## Show the result of the first 17 examples

In [ ]:
for i in range(17):
    print(clean_string(site_mate[i]))
    pred, pred_moves, true_move, board, is_checkmate, is_mate_5_top, legal_move_cnt, mate_move_cnt = eval_pos(net, x_mate[i], yp_mate[i], verbose=True)
    pred_move = pred_moves[0]
    pred_arrow = chess.svg.Arrow(pred_move.from_square, pred_move.to_square)
    SVG(data=chess.svg.board(board=board, arrows=[pred_arrow], size=400))

## Show examples where it failed

In [ ]:
for i in range(1000):
    pred, pred_moves, true_move, board, is_checkmate, is_mate_5_top, legal_move_cnt, mate_move_cnt = eval_pos(net, x_mate[i], yp_mate[i], verbose=False)
    if is_mate_5_top is False:
        print(clean_string(site_mate[i]))
        pred, pred_moves, true_move, board, is_checkmate, is_mate_5_top, legal_move_cnt, mate_move_cnt = eval_pos(net, x_mate[i], yp_mate[i], verbose=True)
        pred_move = pred_moves[0]
        pred_arrow = chess.svg.Arrow(pred_move.from_square, pred_move.to_square)
        SVG(data=chess.svg.board(board=board, arrows=[pred_arrow], size=400))